In [1]:
import os
import cv2
import zipfile
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
def make_submission(result, image_size, stride, save_dir='./ensemble'):
    os.makedirs(os.path.join(save_dir), exist_ok=True)
    os.chdir(os.path.join(save_dir))
    sub_imgs = []
    for i, img in enumerate(result):
        path = f'{20000+i}.png'
        cv2.imwrite(path, img)
        sub_imgs.append(path)
    submission = zipfile.ZipFile(f"submission_ensemble.zip", 'w')
    for path in sub_imgs:
        submission.write(path)
    submission.close()

### 최종 Ensemble 목록

- subm_dirs 구성: (EXP_NAME, SUBMISSION_NAME)
- SUBMISSION_NAME: EXP_NAME/submission/SUBMISSION_NAME.zip 에서 압축해제한 디렉토리명
    - 즉 SUBMISSION_NAME을 넣으면 됨(.zip 파일명에 포함되지 않으니 확인)
    - 압축 해제가 안 되어 있어도 하나하나 압축 해제할 필요 없이 아래 코드에서 압축 해제 진행


In [3]:
subm_dirs = [
    ('10030621_swinir_epoch36_lr1e5', 'submission_img2048_s512_10030621_swinir_epoch36_lr1e5-5'),  # 25.8615023148
    ('10051136_swinir_no-acti_epoch26_lr1e5', 'submission_img2048_s512_10051136_swinir_no-acti_epoch26_lr1e5-10'),
    ('10051136_swinir_no-acti_epoch26_lr1e5', 'submission_img2048_s512_10051136_swinir_no-acti_epoch26_lr1e5-11'),
    ('10061251_swinir_epoch26_l1l2', 'submission_img2048_s512_10061251_swinir_epoch26_l1l2-4'),  # 25.9834354904
    ('10061251_swinir_epoch26_l1l2', 'submission_img2048_s512_10061251_swinir_epoch26_l1l2-5'),
            ]

### Zip파일만 있는 경우 unzip 수행

SUBMISSION 디렉토리가 존재하지 않는 경우 SUBMISSION.zip 압축 해제 진행

In [4]:
for exp_dir, subm_dir in tqdm(subm_dirs):
    if not os.path.exists(os.path.join('experiments', exp_dir, 'submission', subm_dir)):
        with zipfile.ZipFile(os.path.join('experiments',exp_dir, 'submission', f'{subm_dir}.zip'), 'r') as zip_ref:
            zip_ref.extractall(os.path.join('experiments',exp_dir, 'submission', f'{subm_dir}'))

  0%|          | 0/5 [00:00<?, ?it/s]

### Ensemble 진행

In [5]:
result = []
for img_name in tqdm(range(20000, 20018)):
    temp = np.zeros((2048, 2048, 3))
    for exp_dir, subm_dir in subm_dirs:
        img = cv2.imread(os.path.join('experiments', exp_dir, 'submission', subm_dir, f'{img_name}.png'))
        temp += img
    temp /= len(subm_dirs)
    temp = temp.astype(np.uint8)
    result.append(temp)

  0%|          | 0/18 [00:00<?, ?it/s]

In [6]:
make_submission(result, 2048, 512)